# Getting the TOTAL DEATHS from COVID-19 of the 24 departments of Peru

## 0. Run libraries
Empezamos cargando las librerías que necesitamos y leyendo nuestro csv actualizado de fallecidos por COVID-19.

In [48]:
import pandas as pd
import numpy as np

import functions as fn

## 1.

In [ ]:
def just_cities(fal_url):
    """
    Función para filtrar a cada fallecido reportado en su año
    y semana epidemiológica en ciudad (Fallecidos = fal, Vacunados = vac)
    """     
    # Seleccionamos solo las col 'FECHA_FALLECIMIENTO' y 'DEPARTAMENTO'      
    fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO']   
    df_fal = fn.read_largeCSV_file(fal_url, ';', fal_col)
    fn.variable_fecha(df_fal, 'FECHA_FALLECIMIENTO')
    fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO')
    df_fal['fallecido'] = 1                                         
    df_fal['fallecido'].apply(np.int8) 
    df_fal.info()
    
    return df_fal


fal_url = "RawData/fallecidos_covid.csv"
df = just_cities(fal_url)
del fal_url

In [ ]:
print(df.head())

Realizamos un crosstab de del total de fallecidos por COVID-19 por departamento del Perú, por cada semana y año epidemiológico.

In [ ]:
epi_ciudades = pd.crosstab(index=[df['epi_year'], df['epi_week']],
                           columns=[df['fallecido'], df['DEPARTAMENTO']],
                           margins = True)

epi_ciudades.to_csv('Data/fallecidosXciudadesXsemanasEpi.csv')
                           

Ahora realizamos un crosstab del TOTAL de fallecidos por departamento.

In [ ]:
fal_ciudades = pd.crosstab(index = df['DEPARTAMENTO'],
                           columns = df['fallecido'],
                           margins = True)
del fal_ciudades["All"]
fal_ciudades.columns = ['fallecidos']
fal_ciudades.rename(index={'All': 'PERÚ'})

In [ ]:
# https://es.wikipedia.org/wiki/Anexo:Departamentos_del_Per%C3%BA_por_poblaci%C3%B3n
dict_dep = {
    "AMAZONAS"	:	426806,
    "ANCASH"	:	1180638,
    "APURIMAC"	:	430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:	668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LALIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADREDEDIOS":	173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SANMARTIN"	:	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
    "PERÚ"  	:	32625948,
}



no_habitantes = [426806,
                 1180638,
                 430736,
                 1497438,
                 668213,
                 1453711,
                 1129854,
                 1357075,
                 365317,
                 760267,
                 975182,
                 1361467,
                 2016771,
                 1310785,
                 10628470,
                 1027559,
                 173811,
                 192740,
                 271904,
                 2047954,
                 1237997,
                 899648,
                 370974,
                 251521,
                 589110]

no_habitantes.append(sum(no_habitantes))
fal_ciudades['no_habitantes'] = no_habitantes
del no_habitantes

fal_ciudades['tasa_mortalidad'] = (fal_ciudades['fallecidos']/fal_ciudades['no_habitantes'])*100

fal_ciudades.to_csv('Data/TOTAL_fallecidosXciudades.csv')

# Getting the total of people FULLY VACCINATED (2 doses) by the 24 departments of Peru

There are no direct way to found the total of people fully vaccinated per departments. To achieve this the following is planned:

1. The vaccination dataset **(RawData/TB_VACUNACION_COVID19.csv)** only gives information about the vaccination center called *id_centro_vacunacion*. NOT the department or another relevant location.

2. The vaccination centers dataset **(RawData/TB_CENTRO_VACUNACION.csv)** can be used to "match" the *id_centro_vacunacion* with the *id_ubigeo*. Which is a numeric variable from 0 to 1894 of the specifict district.

3.  Finally with the UBIGEO dataset **(RawData/TB_UBIGEOS.csv)** it is possible to "match" each one with the correct department of Peru.


In [52]:
def vac_department(vac_url):
    """
    Función que toma la dirección del  dataset de vacunados y devuelve el número de VACUNADOS 
    por los 24 departamentos del Perú

    Parameters
    ----------
    vac_url: Directorio o url del dataset de VACUNADOS contra COVID-19

    Returns
    -------
    df_vac: Dataframe con los 24 departamentos del Perú y su total de vacunados

    """
    vac_col = ['id_centro_vacunacion', 'dosis']  # Seleccionar solamente la columna del id de los centros de vac.                        
    
    df_vac = fn.read_largeCSV_file(vac_url, ',', vac_col)   # Leemos los datasets     
    lst_vac = fn.df_into_chunks(df_vac)   #  Dividimos en chunks el dataset de vacunados               
    
    for chunk in lst_vac:                                           
        chunk = fn.dose(chunk, "dosis")

    return lst_vac

vac_url = "RawData/TB_VACUNACION_COVID19.csv"
vac_x_depa = vac_department(vac_url)
del vac_url



Read csv with dask:  41.755465269088745 sec
<class 'pandas.core.frame.DataFrame'>
Int64Index: 34638125 entries, 0 to 544332
Columns: 2 entries, id_centro_vacunacion to dosis
dtypes: int64(1), int8(1)
memory usage: 561.6 MB


AttributeError: module 'functions' has no attribute 'dose'

In [51]:
data = {"a":[1,2,3,4],"b":[0,0,0,0]}
df = pd.DataFrame(data)

df = df.drop(df[df["a"] == 3].index)

In [50]:
print(vac_x_depa[0])

        id_centro_vacunacion  dosis
0                         17      2
1                       1828      2
2                     103617      2
3                        891      2
4                     105859      1
...                      ...    ...
499995                   475      2
499996                  4214      2
499997                   767      1
499998                  3789      1
499999                  2681      2

[500000 rows x 2 columns]
